In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import dlc_bci as bci
from utility import * 

## Get Data
Fetches the data from the server and saves it to a folder at the root 

In [3]:
train_input, train_target, test_input, test_target = import100HzData()
train_input_1000, train_target_1000, test_input_1000, test_target_1000 = import1000HzData()

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])
<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [4]:
print(train_input.shape) 
print(train_target.shape) 
print(test_input.shape) 
print(test_target.shape) 

torch.Size([316, 28, 50])
torch.Size([316])
torch.Size([100, 28, 50])
torch.Size([100])


## Visualize the data

In [5]:
inputlen = np.array(train_input[0, :, 0])
print (inputlen.size)

for i in range (0,inputlen.size) :
    time = np.linspace(0, 500, 50)
    data = train_input[0, i, :] #observing the samples n°1 for all time steps and all the electodes 
    data = np.array(data)
    plt.plot(time, data)
    plt.title('Simple plot of electrode n°' + str(i)) 
    plt.xlabel('time (ms)')
    plt.ylabel('Voltage (µV)')
    plt.show() #enables to show all electrodes separately 
    
for i in range (0,inputlen.size) :
    time = np.linspace(0, 500, 50)
    data = train_input[0, i, :]
    data = np.array(data)
    plt.plot(time, data)
    plt.xlabel('time (ms)')
    plt.ylabel('Voltage (µV)')
    plt.title('Simple plot of all electrodes') 

28


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Preprocessing

Noise removed; Trend Removed; Offset translated; Normalized 

### A "0" Labelled signal 

In [6]:
for i in range (0,inputlen.size) :
    time = np.linspace(0, 500, 50)
    data = train_input[0, i, :]
    data = np.array(data)
    plt.plot(time, data)
    plt.xlabel('time (ms)')
    plt.ylabel('Voltage (µV)')
    plt.title('Before Preprocessing - Simple plot of all electrodes') 
plt.show()  

<IPython.core.display.Javascript object>

In [7]:
# GET THE RAW DATA AGAIN 
import matplotlib.pyplot as plt
import numpy as np 
from scipy import signal
import scipy
from scipy.signal import find_peaks_cwt

inputlen = np.array(train_input[0, :, 0]) # This signal is labelled 0 !!!!!!! 
time = np.linspace(0, 500, 50)

welchSpectralEnergy = np.zeros((inputlen.size, 26))
f = np.zeros((inputlen.size, 26))
relmaxValue = np.zeros((inputlen.size, 5))
relmaxTime = np.zeros((inputlen.size, 5))

print (inputlen.size)
    
# Actual Preprocessing set 
for i in range (0,inputlen.size):
    data = train_input[0, i, :]
    data = np.array(data)
    
    f[i,:], welchSpectralEnergy[i, :]=signal.welch(data)
    
    fft=scipy.fft(data) #signal denoising 
    bp=fft[:]
    for j in range(len(bp)): 
        if j>=10:
            bp[j]=0
    ibp=scipy.ifft(bp) 
    
    ibp = signal.detrend(ibp) #signal detrending
    
    #ibp = (ibp-ibp[0])/max(max(ibp), abs(min(ibp))) #signal normalization with initial offset suprresion 
    ibp = (ibp-np.mean(ibp))/np.std(ibp) #signal normalization with initial offset suprresion 
    
    #Find the local maxima of the model (times of the local maxima actually)
    relmax = signal.argrelmax(ibp)
    #print(relmax[0].size)
    if (relmax[0].size == 5): # !!!!! THERE ARE NOT ALWAYS 5 MAXIMA BUT HOW TO SET THE BEST VALUE ???? 
        relmaxTime[i,:] = time[relmax]
        relmaxValue[i,:] = ibp[relmax]
    
    plt.plot(time, ibp)
    plt.xlabel('time (ms)')
    plt.ylabel('Voltage (µV)')
    plt.title('After Preprocessing - Simple plot of all electrodes')
plt.show()

28


/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/scipy/signal/spectral.py:1785: UserWarning: nperseg = 256 is greater than input length  = 50, using nperseg = 50
  .format(nperseg, input_length))
/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/ipykernel_launcher.py:42: ComplexWarning: Casting complex values to real d

<IPython.core.display.Javascript object>

### A "1" Labelled signal 

In [8]:
for i in range (0,inputlen.size) :
    time = np.linspace(0, 500, 50)
    data = train_input[1, i, :]
    data = np.array(data)
    plt.plot(time, data)
    plt.xlabel('time (ms)')
    plt.ylabel('Voltage (µV)')
    plt.title('Before Preprocessing - Simple plot of all electrodes') 
plt.show()

/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<IPython.core.display.Javascript object>

In [9]:
# GET THE RAW DATA AGAIN 
import matplotlib.pyplot as plt
import numpy as np 
from scipy import signal
import scipy

inputlen = np.array(train_input[1, :, 0]) # This signal is labelled 1 !!!!!!! 
print (inputlen.size)

welchSpectralEnergy = np.zeros((inputlen.size, 26))
f = np.zeros((inputlen.size, 26))
    
# Actual Preprocessing set 
for i in range (0,inputlen.size) :
    time = np.linspace(0, 500, 50)
    data = train_input[1, i, :]
    data = np.array(data)
    
    f[i,:], welchSpectralEnergy[i, :]=signal.welch(data)

    fft=scipy.fft(data) #signal denoising 
    bp=fft[:]
    for i in range(len(bp)): 
        if i>=10:
            bp[i]=0
    ibp=scipy.ifft(bp) 

    ibp = signal.detrend(ibp) #signal detrending
    
    #ibp = (ibp-ibp[0])/max(max(ibp), abs(min(ibp))) #signal normalization with initial offset suprresion 
    ibp = (ibp-np.mean(ibp))/np.std(ibp)
    
    plt.plot(time, ibp)
    plt.xlabel('time (ms)')
    plt.ylabel('Voltage (µV)')
    plt.title('After Preprocessing - Simple plot of all electrodes')  
plt.show()

28


/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/scipy/signal/spectral.py:1785: UserWarning: nperseg = 256 is greater than input length  = 50, using nperseg = 50
  .format(nperseg, input_length))
/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/davidcleres/anaconda/envs/DeepLearning/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


<IPython.core.display.Javascript object>

# Neural Net

In [10]:
# Copy the neural network from the Neural Networks section before and modify it to take 18-channel signal (instead of 1-channel images as it was defined).

2. Define a Convolution Neural Network

In [62]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (1,5)) #nn.Conv2d(nbChannels, outputChannels, kxk Square convolution)
        self.pool = nn.MaxPool2d(1, 1)
        self.conv2 = nn.Conv2d(6, 16, (1,5))
        self.fc1 = nn.Linear(16 * 28 * 42, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) #2 output channel one for the left direction and one for the rightweb

    def forward(self, x):
        print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        print(x.shape)
        x = x.view(-1, 16 * 28 * 42)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

3. Define a Loss function and optimizer
Let’s use a Classification Cross-Entropy loss and SGD with momentum.

In [63]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

4. Train the network
This is when things start to get interesting. We simply have to loop over our data iterator, and feed the inputs to the network and optimize.

In [64]:
'''for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        # wrap them in Variable
        inputs, labels = Variable(train_input), Variable(train_target)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')'''

"for epoch in range(2):  # loop over the dataset multiple times\n\n    running_loss = 0.0\n    for i, data in enumerate(trainloader, 0):\n        # get the inputs\n        # wrap them in Variable\n        inputs, labels = Variable(train_input), Variable(train_target)\n\n        # zero the parameter gradients\n        optimizer.zero_grad()\n\n        # forward + backward + optimize\n        outputs = net(inputs)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n\n        # print statistics\n        running_loss += loss.data[0]\n        if i % 2000 == 1999:    # print every 2000 mini-batches\n            print('[%d, %5d] loss: %.3f' %\n                  (epoch + 1, i + 1, running_loss / 2000))\n            running_loss = 0.0\n\nprint('Finished Training')"

In [65]:
'''X_train = np.random.rand(100, 1, 120, 64).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(100, 1, 120, 64).astype('float32')
y_val = np.round(np.random.rand(100).astype('float32'))

X_test = np.random.rand(100, 1, 120, 64).astype('float32')
y_test = np.round(np.random.rand(100).astype('float32'))'''

X_train = train_input
y_train = train_target
X_test = test_input
y_test = test_target

In [67]:
batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)

        
    inputs = X_train
    inputs = torch.unsqueeze(inputs,1)
    labels = y_train

    # wrap them in Variable
    inputs, labels = Variable(inputs), Variable(labels)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
        
        
    optimizer.step()

    running_loss += loss.data[0]
    print(running_loss)
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]


Epoch  0
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
7.800505340099335

Epoch  1
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
8.470728635787964

Epoch  2
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
9.124361991882324

Epoch  3
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
9.786439538002014

Epoch  4
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
10.433080554008484

Epoch  5
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
11.085434794425964

Epoch  6
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
11.724815547466278

Epoch  7
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])
torch.Size([316, 16, 28, 42])
12.367925643920898

Epoch  8
torch.Size([316, 1, 28, 50])
torch.Size([316, 6, 28, 46])


5. Test the network on the test data
We have trained the network for 2 passes over the training dataset. But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network outputs, and checking it against the ground-truth. If the prediction is correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.

In [68]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

NameError: name 'testloader' is not defined

In [29]:
inputs.TYPE


AttributeError: 'torch.FloatTensor' object has no attribute 'TYPE'